In [22]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('data.csv')

print(data.columns)

# Replace '?' with NaN for missing value handling
data.replace('?', np.nan, inplace=True)

# Convert numerical columns to float where applicable
# Exclude 'num' column from initial numeric conversion
numeric_columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                   'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Impute missing values with the median of respective columns
data.fillna(data.median(), inplace=True)

# Check if 'num' column exists before converting
if 'num' in data.columns:
    # Now convert 'num' column to numeric
    data['num'] = pd.to_numeric(data['num'], errors='coerce')

    # Binarize the target variable (heart attack: 0 for no, 1 for yes)
    data['num'] = data['num'].apply(lambda x: 1 if x > 0 else 0)
else:
    print("Warning: 'num' column not found in the dataset.")
    # You might want to handle this case differently based on your specific needs.
    # For example, you could create a new 'num' column with default values.

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num', 'num1'],
      dtype='object')


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Features and target variable
X = data.drop(columns=['num'])
y = data['num']

# Scale features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape for LSTM and Transformer
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train LSTM model
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - accuracy: 0.5427 - loss: 0.6921 - val_accuracy: 0.6441 - val_loss: 0.6871
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6525 - loss: 0.6852 - val_accuracy: 0.6441 - val_loss: 0.6798
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6531 - loss: 0.6757 - val_accuracy: 0.6441 - val_loss: 0.6695
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6549 - loss: 0.6660 - val_accuracy: 0.6441 - val_loss: 0.6553
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6533 - loss: 0.6519 - val_accuracy: 0.6441 - val_loss: 0.6294
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6447 - loss: 0.6264 - val_accuracy: 0.7458 - val_loss: 0.5661
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7038 - loss: 0.5828 - val_accuracy: 0.7966 - val_loss: 0.4908
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7939 - loss: 0.4881 - val_accuracy: 0.7966 - val_loss: 0.3979
Epoch 9/50


In [25]:
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.models import Model

# Transformer model
input_layer = Input(shape=(X_train.shape[1], 1))
attention = MultiHeadAttention(num_heads=4, key_dim=4)(input_layer, input_layer)
norm = LayerNormalization()(attention)
dense = Dense(64, activation='relu')(norm)
global_pool = GlobalAveragePooling1D()(dense)
output = Dense(1, activation='sigmoid')(global_pool)

transformer_model = Model(inputs=input_layer, outputs=output)

# Compile Transformer model
transformer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Transformer model
transformer_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6078 - loss: 0.6930 - val_accuracy: 0.6441 - val_loss: 0.6923
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6107 - loss: 0.6922 - val_accuracy: 0.6441 - val_loss: 0.6913
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5895 - loss: 0.6918 - val_accuracy: 0.6441 - val_loss: 0.6903
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6309 - loss: 0.6903 - val_accuracy: 0.6441 - val_loss: 0.6893
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6303 - loss: 0.6895 - val_accuracy: 0.6441 - val_loss: 0.6884
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6113 - loss: 0.6893 - val_accuracy: 0.6441 - val_loss: 0.6876
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6267 - loss: 0.6881 - val_accuracy: 0.6441 - val_loss: 0.6867
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6723 - loss: 0.6852 - val_accuracy: 0.6441 - val_loss: 0.6859
Epoch 

In [26]:
# Evaluate LSTM Model
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Model - Loss: {lstm_loss}, Accuracy: {lstm_accuracy}")

# Evaluate Transformer Model
transformer_loss, transformer_accuracy = transformer_model.evaluate(X_test, y_test)
print(f"Transformer Model - Loss: {transformer_loss}, Accuracy: {transformer_accuracy}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0758 
LSTM Model - Loss: 0.06839817017316818, Accuracy: 1.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6481 - loss: 0.6624 
Transformer Model - Loss: 0.6634902358055115, Accuracy: 0.6440678238868713
